In [ ]:
## Text cleaning and similarity workflow
# - Load `apt.txt`, `paper2code.txt`, and `protopy.txt`
# - Strip whitespace/punctuation noise and normalize casing
# - Tokenize cleaned strings and summarize token counts
# - Compute cosine similarity across the token-count vectors



In [ ]:
from __future__ import annotations

from pathlib import Path
import re
from collections import Counter
from typing import Dict, List

import numpy as np
import pandas as pd

BASE_DIR = Path("/Users/v1p0r/Documents/DDL_Fall25/term_project/protopy_agent/eval")
SOURCE_FILES: Dict[str, Path] = {
    "apt": BASE_DIR / "apt.txt",
    "paper2code": BASE_DIR / "paper2code.txt",
    "protopy": BASE_DIR / "protopy.txt",
}

_non_word_pattern = re.compile(r"[^\w]+", re.UNICODE)
_multi_space_pattern = re.compile(r"\s+")

def load_and_clean(path: Path) -> str:
    """Load text, lower-case it, drop punctuation/newlines, and collapse spaces."""
    raw = path.read_text(encoding="utf-8", errors="ignore")
    lowered = raw.lower()
    # Replace any non-alphanumeric/underscore stretches with a single space
    alnum_only = _non_word_pattern.sub(" ", lowered)
    collapsed = _multi_space_pattern.sub(" ", alnum_only).strip()
    return collapsed

def tokenize(clean_text: str) -> List[str]:
    """Tokenize on single spaces (already normalized)."""
    if not clean_text:
        return []
    return clean_text.split(" ")

def counter_to_vector(counter: Counter, vocab: List[str]) -> np.ndarray:
    return np.array([counter.get(tok, 0) for tok in vocab], dtype=float)

def cosine_similarity_matrix(vectors: np.ndarray) -> np.ndarray:
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    norms[norms == 0] = 1.0  # avoid division by zero for empty docs
    normalized = vectors / norms
    return normalized @ normalized.T



In [ ]:
cleaned_texts = {name: load_and_clean(path) for name, path in SOURCE_FILES.items()}

cleaning_summary = pd.DataFrame([
    {
        "file": name,
        "characters_after_cleaning": len(text),
        "preview": text[:140] + ("…" if len(text) > 140 else "")
    }
    for name, text in cleaned_texts.items()
])

cleaning_summary



In [ ]:
tokenized_texts = {name: tokenize(text) for name, text in cleaned_texts.items()}
token_counters = {name: Counter(tokens) for name, tokens in tokenized_texts.items()}

token_summary = pd.DataFrame([
    {
        "file": name,
        "token_count": len(tokenized_texts[name]),
        "unique_tokens": len(token_counters[name]),
        "sample_tokens": tokenized_texts[name][:15],
    }
    for name in tokenized_texts
])

token_summary



In [ ]:
vocab = sorted({token for tokens in tokenized_texts.values() for token in tokens})
if not vocab:
    raise ValueError("No tokens found after cleaning; cannot compute similarity.")

vectors = np.vstack([
    counter_to_vector(token_counters[name], vocab) for name in tokenized_texts
])
cosine_matrix = cosine_similarity_matrix(vectors)

similarity_df = pd.DataFrame(
    cosine_matrix,
    index=list(tokenized_texts.keys()),
    columns=list(tokenized_texts.keys()),
)

pairwise_targets = [("apt", "paper2code"), ("apt", "protopy")]
pairwise_similarity = pd.DataFrame(
    [
        {
            "pair": f"{a} vs {b}",
            "cosine_similarity": float(similarity_df.loc[a, b]),
        }
        for a, b in pairwise_targets
    ]
)

pairwise_similarity



In [9]:
import mermaid as md
from mermaid.graph import Graph

render = md.Mermaid("""
---
config:
  theme: neutral
  look: handDrawn
  layout: dagre
title: Paper-to-Code System Overview
---
flowchart TB
 subgraph SFT["SFT Dataset"]
        F("⬅️ Input")
        G("➡️ Output")
  end
    A("📄 Arxiv Papers") --> B("🧾 Structured<br>Papers (JSON)")
    B -- 🤖 DeepSeek --> C("📈 Paper with Model Imrprovment")
    C -- 🤖 DeepSeek --> D["📑 Training Template <br> (JSON)"]
    C -- 🔍 RegEx --> E["🧩 Model Definition <br> Code"]
    D --> F
    E --> G
""")

In [10]:
render